In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import seaborn as sns
import src.utils
import seaborn as sns
import matplotlib.pyplot as plt

from src.evaluation import (
    evaluate,
    encode_sequences,
    encode_sequence
    )
import src.config

import src.config
import src.data
import src.model_new
import src.utils
from src.model_new import (
    T5EncoderModelForTokenClassification,
)

import gc
import copy
import random

from transformers import (
    T5Tokenizer,
    DataCollatorForTokenClassification,
    TrainingArguments,
    Trainer,
    TrainerCallback
)

import peft
from peft import (
    LoraConfig,
)

In [ ]:
ROOT = src.utils.get_project_root_path()
device = torch.device('cuda:0' if torch.cuda.is_available() else ('mps' if torch.backends.mps.is_available() else 'cpu'))

USE_CRF = True
EXPERT = 'SP'

SEED = 42
torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

print("Base Model:\t", src.config.base_model_name)
print("MPS:\t\t", torch.backends.mps.is_available())
print("Path:\t\t", ROOT)
print(f"Using device:\t {device}")

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
FASTA_FILENAME = '5_SignalP_5.0_Training_set_testing.fasta'
df_data = src.data.process(src.data.parse_file(ROOT + '/data/raw/' + FASTA_FILENAME))
df_data = df_data[df_data['Partition_No'] == 4].reset_index(drop=True)
df_data['Sequence'] = df_data['Sequence'].apply(lambda x: x.replace(' ', ''))

In [ ]:
df_data['Prediction'] = df_data['Label'].apply(lambda x: x)

In [ ]:
df_data

In [ ]:
df_data[df_data['Type'] == 'LIPO']['Prediction'] = df_data[df_data['Type'] == 'LIPO']['Prediction'].apply(lambda x: 1)

In [ ]:
df_data

In [ ]:
# [len(x) for x in df_data['Prediction'].value_counts().index]

In [ ]:
# targets = [
#     'SSSOTLTMLLLOOTSOOOLMTLMOLTLLMMLLLSLLSSSSOOOTTMSIOMITTITIOIITOTOTTLSTOTM',
#     'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIMMMMMMMMIIIIIIIIIIIIMIIIMMIIIMIIIL',
#     'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIMMMMMMMMIIIIIIIIIIIIMIIIMMIIIMIIIL'
#     ]
# predictions = [
#     'SSSOTLTMLLLOOTSOOOLMTLMOLTLLMMLLLSLLSSSSOOOTTMSIOMITTITIOIITOTOTTLSTOTM',
#     'MIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIMMMMMMMMIIIIIIIIIIIIMIIIMMIIIMIIIL',
#     'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIMMMMMMMMIIIIIIIIOSTIMIIIMMIIIMIIIL'
#     ]


# predictions = encode_sequences(predictions, encoding=src.config.label_encoding)
# targets = encode_sequences(targets, encoding=src.config.label_encoding)

In [ ]:
target =        'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIMMMMMMMMIIIIIIIIIIIIMIIIMMIIIMIIIL'
prediction =    'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIMMMMMMMMIIIIIIIIOSTIMIIIMMIIIMIIIL'

target = encode_sequence(target, encoding=src.config.label_encoding)
prediction = encode_sequence(prediction, encoding=src.config.label_encoding)

In [ ]:
evaluation = evaluate(
    predictions=prediction,
    targets=target,
    encoding=src.config.label_encoding,
    decoding=src.config.label_decoding
)

In [ ]:
df_evaluation = pd.DataFrame(evaluation)

In [ ]:
df_evaluation_MCC = pd.DataFrame(df_evaluation['MCC_2'].to_list(), columns=src.config.label_encoding)
df_evaluation_MCC_mean = pd.DataFrame(np.nanmean(df_evaluation_MCC, axis=0).reshape(-1, 6), columns=src.config.label_encoding)
df_evaluation_MCC_dev = pd.DataFrame(np.std(df_evaluation_MCC, axis=0)).T
df_evaluation_MCC_error = pd.DataFrame(np.std(df_evaluation_MCC, axis=0, ddof=1)).T
display(df_evaluation_MCC, df_evaluation_MCC_mean, df_evaluation_MCC_error)
# display(df_evaluation_MCC)

In [ ]:
src.evaluation.plot_mcc(df_evaluation_MCC)

In [ ]:
src.evaluation.plot_confusion_matrix(df_evaluation['CM'].sum(), src.config.label_decoding)

In [ ]:
FASTA_FILENAME = '5_SignalP_5.0_Training_set.fasta'
# FASTA_FILENAME = '5_SignalP_5.0_Training_set_testing.fasta'
df_data = src.data.process(src.data.parse_file(ROOT + '/data/raw/' + FASTA_FILENAME))

In [ ]:
df_data.shape

In [ ]:
df_data[df_data['Sequence'].str.startswith('M A A V I L E R L G A L W V Q N L R G K L A L G I L P Q')]

In [ ]:
'M M M M M M M M M M M M M M M M M M M M M M M M M M M M M M M M M M M M M M M M M M M M M M M M M M M M M M M M M M M M M M M M M M M M M M M'.replace(' ', '')

In [ ]:
df_data